<span style='color:darkcyan'><font size="3">__Juan Sebastian Deslarzes__</font></span><br>
<span style='color:darkcyan'><font size="3">__Chloé De Ancos__</font></span><br>
<span style='color:darkcyan'><font size="3">__Anaïs Burrus__</font></span><br>
<span style='color:darkcyan'><font size="3">__Margot Chapot__</font></span><br>
<span style='color:darkcyan'><font size="3">__Eva Sarlin__</font></span><br>



<span style='color:darkcyan'><font size="8">__Notebook 2 : Concatenation Method__</font></span><br>

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import numpy as np
from skimage import io
from IPython import display
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
import seaborn as sns
sns.set(color_codes=True)

Loading BokehJS ...

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from collections import Counter 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
import string

from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evasarlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<span style='color:mediumpurple'><font size="6">__Data Loading__</font></span><br>

In [3]:
sample_submission_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/sample_submission.csv'
test_nolabel_url= 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/test_nolabel.csv'
train_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/train.csv'

sample_submission= pd.read_csv(sample_submission_url)
test_nolabel = pd.read_csv(test_nolabel_url)
train = pd.read_csv(train_url)

<span style='color:mediumpurple'><font size="6">__Data Visualization__</font></span><br>

In [4]:
train.head()

,id,label,statement,subject,speaker,speaker_job,state_info,party_affiliation
0,81f884c64a7,1,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican
1,30c2723a188,0,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat
2,6936b216e5d,0,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat
3,b5cd9195738,1,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,NaN,NaN,none
4,84f8dac7737,0,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,NaN,Wisconsin,republican


In [5]:
print('Data train dimensions : ', train.shape)
print('Unique values for label : ', train['label'].unique())
print("Number of unique values for statement: ", train['statement'].unique().size)
print("Number of unique values for subject: ", (train['subject'].unique()).size)
print("Number of unique values for speaker: ", (train['speaker'].unique()).size)
print("Number of unique values for speaker job: ", (train['speaker_job'].unique()).size)
print("Number of unique values for state information: ", (train['state_info'].unique()).size)
print("Number of unique values for party affiliation: ", (train['party_affiliation'].unique()).size)

Data train dimensions :  (8950, 8)
Unique values for label :  [1 0]
Number of unique values for statement:  8939
Number of unique values for subject:  3409
Number of unique values for speaker:  2634
Number of unique values for speaker job:  1092
Number of unique values for state information:  80
Number of unique values for party affiliation:  24


<div class="alert alert-block alert-info">
    <span style='color:darkblue'><font size="4"><b>CONCLUSION DATA VISUALIZATION</b></font></span><br><br> <span style='color:black'> As we can see we have a training set of 8950 instances, with 7 columns. We face a classification problem with the target being the column "label" taking as values 0 or 1. We have 6 features, each of them having a large number of possible values. This is due to the fact that these features contain strings, generating a large number of possibilities. The challenge is going to find a way to process these strings in order to use them as input of classification machine learning algorithms. In this notebook we will focus on one of the 3 methods we found, this one being called the "concatenation" method. </span>
</div>

<span style='color:mediumpurple'><font size="6">__Feature Selection and Cleaning__</font></span><br>

In [6]:
train['speaker_job']= train['speaker_job'].fillna('u.s. senator')
test_nolabel['speaker_job']= test_nolabel['speaker_job'].fillna('u.s. senator')

train['state_info']= train['state_info'].fillna('unknown')
test_nolabel['state_info']= test_nolabel['state_info'].fillna('unknown')

In [7]:
cvec=CountVectorizer()

X1 = train['statement']
y = train['label']
X_test1 = test_nolabel['statement']

X2 = train['subject']
X_test2 = test_nolabel['subject']

X3 = train['speaker']
X_test3 = test_nolabel['speaker']

X4 = train['speaker_job']
X_test4 = test_nolabel['speaker_job']

X5 = train['state_info']
X_test5 = test_nolabel['state_info'] 

X6 = train['party_affiliation']
X_test6 = test_nolabel['party_affiliation']

<span style='color:mediumpurple'><font size="6">__NLP__</font></span><br>

<span style='color:steelblue'><font size="3">__The idea is to create dataframes for train and test for each features and to concatenate them at the end with the aim to pass the total dataframe to a classifier. As the classifier will only accept numerical values we need to vectorize the content of each features.__</font></span><br>

In [8]:
cvec=CountVectorizer(stop_words='english').fit(X1)
statement_train=pd.DataFrame(cvec.transform(X1).todense(),columns=cvec.get_feature_names_out())
statement_test=pd.DataFrame(cvec.transform(X_test1).todense(),columns=cvec.get_feature_names_out())

In [9]:
cvec=CountVectorizer(stop_words='english').fit(X2)
subject_train= pd.DataFrame(cvec.transform(X2).todense(),columns=cvec.get_feature_names_out())
subject_test= pd.DataFrame(cvec.transform(X_test2).todense(),columns=cvec.get_feature_names_out())

In [10]:
cvec=CountVectorizer(stop_words='english').fit(X3)
speaker_train= pd.DataFrame(cvec.transform(X3).todense(),columns=cvec.get_feature_names_out())
speaker_test= pd.DataFrame(cvec.transform(X_test3).todense(),columns=cvec.get_feature_names_out())

In [11]:
cvec=CountVectorizer(stop_words='english').fit(X4)
speakerjob_train= pd.DataFrame(cvec.transform(X4).todense(),columns=cvec.get_feature_names_out())
speakerjob_test= pd.DataFrame(cvec.transform(X_test4).todense(),columns=cvec.get_feature_names_out())

In [12]:
cvec=CountVectorizer(stop_words='english').fit(X5)
state_train= pd.DataFrame(cvec.transform(X5).todense(),columns=cvec.get_feature_names_out())
state_test= pd.DataFrame(cvec.transform(X_test5).todense(),columns=cvec.get_feature_names_out())

In [13]:
cvec=CountVectorizer(stop_words='english').fit(X6)
party_train= pd.DataFrame(cvec.transform(X6).todense(),columns=cvec.get_feature_names_out())
party_test= pd.DataFrame(cvec.transform(X_test6).todense(),columns=cvec.get_feature_names_out())

<span style='color:steelblue'><font size="3">__Now we have 6 different pairs of dataframe with the features content vectorized. Let's put them together :__</font></span><br>

In [14]:
total_train=pd.concat([statement_train, subject_train, speaker_train, speakerjob_train, state_train, party_train], axis=1)
total_test=pd.concat([statement_test, subject_test, speaker_test, speakerjob_test, state_test , party_test], axis=1)

In [15]:
print (total_train.shape)
print (total_test.shape)

(8950, 15503)
(3836, 15503)


<span style='color:steelblue'><font size="3">__Now that we have one big dataframe with numerical values we can pass it to a classifier:__</font></span><br>

<span style='color:mediumpurple'><font size="6">__Machine Learning Classification__</font></span><br>

In [16]:
lr=LogisticRegression()

In [17]:
lr.fit(total_train,y)

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

<span style='color:mediumpurple'><font size="6">__Ensemble__</font></span><br>


<span style='color:steelblue'><font size="3">__Testing the accuracy of different machine learning algorithms on a subset of the train :__</font></span><br>

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_train, y, test_size=0.2, random_state=0)

<span style='color:green'><font size="3">__Random Forest:__</font></span><br>

In [26]:
classifier1 = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier1.fit(X_train, y_train) 
print(classification_report(y_test, classifier1.predict(X_test)))

RandomForestClassifier(n_estimators=1000, random_state=0)

              precision    recall  f1-score   support

           0       0.59      0.13      0.21       623
           1       0.67      0.95      0.79      1167

    accuracy                           0.67      1790
   macro avg       0.63      0.54      0.50      1790
weighted avg       0.64      0.67      0.59      1790

0.6653631284916202


<span style='color:green'><font size="3">__Naives Bayes:__</font></span><br>

In [28]:
from sklearn.naive_bayes import GaussianNB
classifier2 = GaussianNB()
classifier2.fit(X_train, y_train)
print(classification_report(y_test, classifier2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.38      0.63      0.47       623
           1       0.69      0.45      0.55      1167

    accuracy                           0.51      1790
   macro avg       0.54      0.54      0.51      1790
weighted avg       0.58      0.51      0.52      1790

0.5128491620111731


<span style='color:green'><font size="3">__K Nearest Neighbors__</font></span><br>

In [30]:
from sklearn.neighbors import KNeighborsClassifier
classifier3 = KNeighborsClassifier(n_neighbors=10)
classifier3.fit(X_train, y_train)
print(classification_report(y_test, classifier3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.46      0.29      0.36       623
           1       0.68      0.82      0.75      1167

    accuracy                           0.63      1790
   macro avg       0.57      0.55      0.55      1790
weighted avg       0.61      0.63      0.61      1790

0.6346368715083799


<span style='color:green'><font size="3">__Decision Tree:__</font></span><br>

In [72]:
from sklearn import tree
classifier4 = tree.DecisionTreeClassifier()
classifier4.fit(X_train, y_train)
print(classification_report(y_test, classifier4.predict(X_test)))

              precision    recall  f1-score   support

           0       0.44      0.40      0.42       623
           1       0.69      0.73      0.71      1167

    accuracy                           0.61      1790
   macro avg       0.57      0.56      0.56      1790
weighted avg       0.61      0.61      0.61      1790

0.6134078212290502


<span style='color:green'><font size="3">__Multi Layers Perceptron:__</font></span><br>

In [32]:
from sklearn.neural_network import MLPClassifier
classifier5 = MLPClassifier(random_state=1, max_iter=300)
classifier5.fit(X_train, y_train)
print(classification_report(y_test, classifier5.predict(X_test)))

              precision    recall  f1-score   support

           0       0.44      0.43      0.43       623
           1       0.70      0.71      0.70      1167

    accuracy                           0.61      1790
   macro avg       0.57      0.57      0.57      1790
weighted avg       0.61      0.61      0.61      1790

0.6094972067039106


<span style='color:green'><font size="3">__Support Vector Classifier SVC:__</font></span><br>

In [34]:
from sklearn.svm import SVC
classifier6 = SVC(gamma='auto')
classifier6.fit(X_train, y_train)
print(classification_report(y_test, classifier6.predict(X_test)))

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       623
           1       0.65      1.00      0.79      1167

    accuracy                           0.65      1790
   macro avg       0.33      0.50      0.39      1790
weighted avg       0.43      0.65      0.51      1790

0.6519553072625698


<div class="alert alert-block alert-info">
    <span style='color:darkblue'><font size="4"><b>CONCLUSION ALGORITHMS</b></font></span><br><br> <span style='color:black'> After testing several algorithms and submit their predictions on kaggle, we found that the regression was the best classifier, giving us an accuracy above 57% whereas Random Frest and SVC were not performing well (51% and 50,5%), even if they showed accuracy of 65% on teh training subset. </span>
</div>


<span style='color:steelblue'><font size="5">__Tuning of hyperparameters using genetic algorithms :__</font></span><br>

In [35]:
param_grid_kNN = [{'n_neighbors': [10, 12, 15], # np.arange(4, 10),
                    # 'weights': ['uniform', 'distance'],
                    'algorithm':['auto', 'ball_tree'] #, 'kd_tree', 'brute']
                            
                    }]

param_grid_mlp = [{'activation': ['logistic','tanh','relu'],
                    'solver':['lbfgs', 'sgd', 'adam']         
                    }]

param_grid_mb = [{'alpha': np.arange(0.1, 0.5),    
                    }]

param_grid_DT = [{'criterion': ['gini', 'entropy'], 
                  'max_leaf_nodes' : np.arange(2,30),
                  'max_depth' : np.arange(2,6)
                    }]

param_grid_RF =  [{ #'bootstrap': [True, False],
                     'max_depth': [15, None],
                    # 'max_features': ['auto', 'sqrt'],
                   #  'min_samples_leaf': [1, 2, 4],
                    # 'min_samples_split': [2, 5, 10],
                     'n_estimators': [200, 400]
                  }]


<span style='color:red'><font size="5">__Disclaimer : takes too much time for you to run__</font></span><br>

In [36]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.model_selection import StratifiedKFold

kNN_GA = EvolutionaryAlgorithmSearchCV(estimator=KNeighborsClassifier(),
                                       params=param_grid_kNN,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time kNN_GA.fit(X_train, y_train)

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Types [1, 1] and maxint [0, 1] detected
--- Evolve in 2 possible combinations ---
gen	nevals	avg     	min     	max     	std       
0  	50    	0.643235	0.642179	0.644693	0.00124079
1  	34    	0.644693	0.644693	0.644693	1.11022e-16
2  	30    	0.644693	0.644693	0.644693	1.11022e-16
3  	35    	0.644592	0.642179	0.644693	0.000492635
4  	30    	0.644592	0.642179	0.644693	0.000492635
5  	27    	0.644642	0.642179	0.644693	0.000351955
6  	27    	0.644693	0.644693	0.644693	1.11022e-16
7  	25    	0.644693	0.644693	0.644693	1.11022e-16
8  	23    	0.644693	0.644693	0.644693	1.11022e-16
9  	26    	0.644693	0.644693	0.644693	1.11022e-16
10 	28    	0.644642	0.642179	0.644693	0.000351955
Best individual is: {'n_neighbors': 15, 'algorithm': 'ball_tree'}
with fitness: 0.6446927374301676
CPU times: user 8min 55s, sys: 8.24 s, total: 9min 3s
Wall time: 9min 6s


In [22]:
RF_GA = EvolutionaryAlgorithmSearchCV(estimator=RandomForestClassifier(),
                                       params=param_grid_RF,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time RF_GA.fit(X_train, y_train)

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Types [1, 1] and maxint [3, 2] detected
--- Evolve in 12 possible combinations ---
gen	nevals	avg     	min     	max     	std       
0  	50    	0.650115	0.646369	0.653631	0.00303939
1  	38    	0.653575	0.652095	0.653631	0.000263519
2  	26    	0.653525	0.648324	0.653631	0.000743017
3  	19    	0.653564	0.652095	0.653631	0.000301158
4  	34    	0.653542	0.652095	0.653631	0.000334916
5  	27    	0.653628	0.653492	0.653631	1.95531e-05
6  	34    	0.653316	0.646369	0.653631	0.00109254 
7  	30    	0.653567	0.652095	0.653631	0.000301119
8  	35    	0.653564	0.652095	0.653631	0.000301158
9  	27    	0.653606	0.652514	0.653631	0.000157245
10 	22    	0.653598	0.652095	0.653631	0.000215573
Best individual is: {'max_depth': None, 'n_estimators': 200}
with fitness: 0.6536312849162011
CPU times: user 32min 47s, sys: 45.4 s, total: 33min 32s
Wall time: 36min 19s


In [ ]:
MLP_GA = EvolutionaryAlgorithmSearchCV(estimator=MLPClassifier(),
                                       params=param_grid_mlp,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time MLP_GA.fit(X_train, y_train)

In [70]:
MB_GA = EvolutionaryAlgorithmSearchCV(estimator=MultinomialNB(),
                                       params=param_grid_mb,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time MB_GA.fit(X_train, y_train)

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Types [2] and maxint [0] detected
--- Evolve in 1 possible combinations ---
gen	nevals	avg     	min     	max     	std        
0  	50    	0.597207	0.597207	0.597207	1.11022e-16
1  	26    	0.597207	0.597207	0.597207	1.11022e-16
2  	29    	0.597207	0.597207	0.597207	1.11022e-16
3  	28    	0.597207	0.597207	0.597207	1.11022e-16
4  	25    	0.597207	0.597207	0.597207	1.11022e-16
5  	39    	0.597207	0.597207	0.597207	1.11022e-16
6  	30    	0.597207	0.597207	0.597207	1.11022e-16
7  	35    	0.597207	0.597207	0.597207	1.11022e-16
8  	31    	0.597207	0.597207	0.597207	1.11022e-16
9  	33    	0.597207	0.597207	0.597207	1.11022e-16
10 	35    	0.597207	0.597207	0.597207	1.11022e-16
Best individual is: {'alpha': 0.1}
with fitness: 0.5972067039106145
CPU times: user 3.96 s, sys: 1.3 s, total: 5.26 s
Wall time: 4.9 s


In [77]:
DT_GA = EvolutionaryAlgorithmSearchCV(estimator=tree.DecisionTreeClassifier(),
                                       params=param_grid_DT,
                                       scoring="accuracy",
                                       cv=StratifiedKFold(n_splits=2),
                                       verbose=True,
                                       population_size=50,
                                       gene_mutation_prob=0.10,
                                       tournament_size=20,
                                       generations_number=10)
%time DT_GA.fit(X_train, y_train)

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Types [1, 1, 1] and maxint [1, 27, 3] detected
--- Evolve in 224 possible combinations ---
gen	nevals	avg     	min     	max     	std        
0  	50    	0.646115	0.645391	0.646927	0.000412132
1  	28    	0.646698	0.64581 	0.646927	0.000273344
2  	26    	0.646927	0.646927	0.646927	1.11022e-16
3  	30    	0.646877	0.645531	0.646927	0.000221288
4  	29    	0.646891	0.64595 	0.646927	0.00016    
5  	33    	0.646891	0.646369	0.646927	0.000127425
6  	30    	0.646902	0.64595 	0.646927	0.000141579
7  	22    	0.646922	0.646648	0.646927	3.91061e-05
8  	30    	0.646927	0.646927	0.646927	1.11022e-16
9  	30    	0.646927	0.646927	0.646927	1.11022e-16
10 	26    	0.646894	0.645531	0.646927	0.000198619
Best individual is: {'criterion': 'gini', 'max_leaf_nodes': 20, 'max_depth': 5}
with fitness: 0.646927374301676
CPU times: user 3min 7s, sys: 1min 4s, total: 4min 12s
Wall time: 4min 13s


<span style='color:steelblue'><font size="5">__Let's combine some of these algoritms in a Voting classifier :__</font></span><br>

In [ ]:
from sklearn.ensemble import VotingClassifier
VotingModel = VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=15)), ('mlp', MLPClassifier(random_state=1, max_iter=300)), ('lr', lr),  ('svc',SVC(gamma='auto')), ('RF', RandomForestClassifier(n_estimators=1000, random_state=0)), ('tree', tree.DecisionTreeClassifier(criterion='gini', max_depth=5,  max_leaf_nodes=20))], voting='hard')
VotingModel.fit(total_train, y)

<span style='color:mediumpurple'><font size="6">__Prediction__</font></span><br>

In [18]:
ypred=lr.predict(total_test) 

In [63]:
ypred=VotingModel.predict(total_test)

<span style='color:mediumpurple'><font size="6">__Submission__</font></span><br>

In [80]:
id = test_nolabel['id']
dfsubmission = pd.DataFrame({'id': id, 'label': ypred}, columns=["id", "label"])
dfsubmission.to_csv('concatenationVoting2.csv', index=False, header=True)

<div class="alert alert-block alert-info">
    <span style='color:purple'><font size="4"><b>CONCLUSION GENERALE</b></font></span><br><br> <span style='color:black'> To sum up, we tried another way to combine the different features. This time the content of each feature was vectorized to obtain a dataframe of numerical values, and the resulting dataframes were concatenated and passed to machine learning classifiers. As machine learning algorithms, we tested several algorithms and concluded that the regression fits the best this implementation. We also used genetic algorithms to tune the hyperparameters using a subset of the provided training set. Finally we used all these algorithms in a Voting Classifier. </span>
</div>